In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint as pp
import json
import pandas as pd
from pandas.api.types import CategoricalDtype
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path
import backtrader as bt
from nomics_bt_strategy import VolumeStrategy
from nomics_data_prep_for_backtrader import load_currency_data, all_currencies
from backtrader.feeds import PandasData
import dill as pickle

class PandasData_normal(PandasData):
    """
    The ``dataname`` parameter inherited from ``bt.feed.DataBase`` is the pandas
    DataFrame
    """
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ("datetime", None),
        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ("open", "open"),
        ("high", "high"),
        ("low", "low"),
        ("close", "close"),
        ("volume", "volume"),
    )
class PandasData_transparent(PandasData):
    """
    The ``dataname`` parameter inherited from ``bt.feed.DataBase`` is the pandas
    DataFrame
    """
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ("datetime", None),
        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ("open", "transparent_open"),
        ("high", "transparent_high"),
        ("low", "transparent_low"),
        ("close", "transparent_close"),
        ("volume", "transparent_volume"),
    )

def type_raw_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    cat_status = CategoricalDtype(
    categories=['closed', 'opened'], 
    ordered=True,
    )
    return_df = df.astype(
    {
        'buy_date': 'datetime64[ns]',
        'sell_date': 'datetime64[ns]',
        'buy_price': 'float64',
        'sale_price': 'float64',
        'currency_id': 'str',
        'run_name': str,
        'status': cat_status,
        'sell_price_multiple': 'int16',
        'buy_volume_multiple': 'int16',
        'volume_avg_duration': 'int16',
        'transparent': bool,

        }, copy=True
    )
    return return_df

In [6]:
results_folder = Path("../bt_data_runs/").resolve()
temp_save_path = results_folder.joinpath(f"_temp_df.pkl")
archive_save_path = results_folder.joinpath(f"archive_df.pkl")

# Option to save existing dataframe and skip already completed currencies
total_redo = False
if total_redo:
    # clear existing archive
    if os.path.exists(archive_save_path):
        os.remove(archive_save_path)
    archive_df = pd.DataFrame()
    completed_currencies = []
else:
    # load existing archive
    if os.path.exists(archive_save_path):
        archive_df = pd.read_pickle(archive_save_path)
        with open(results_folder.joinpath(f"completed_currencies.pkl"), "rb") as f:
            completed_currencies = pickle.load(f)
    else:
        archive_df = pd.DataFrame()
        completed_currencies = []


currencies = all_currencies()
# remove already completed currencies
currencies = [c for c in currencies if c not in completed_currencies]

for currency in progress_bar(currencies, desc="Currencies"):
    # load data
    data_df = load_currency_data(currency)
    data_df.set_index('timestamp', inplace=True)

    data_normal = PandasData_normal(dataname=data_df)
    data_transparent = PandasData_transparent(dataname=data_df)
    df_list = []
    
    candle_length = len(data_df)

    for VolAvgParam in progress_bar(range(5,95,5), desc=f"VolAvgParameters-{currency}", leave=False, position=0):
        if VolAvgParam + 2 >= candle_length:
            # avoid errors with averaging
            continue
        for TriggerVolParam in range(6,20,2):
            for transparent in [True, False]:
                cerebro = bt.Cerebro()
                # Add the Data Feed to Cerebro
                if transparent:
                    cerebro.adddata(data_transparent)
                else:
                    cerebro.adddata(data_normal)
                # params = (
                #         ("VolAvgLength", int(50)),
                #         ("TriggerPriceMultipliers", list(range(4, 22, 2))),
                #         ("TriggerVolumeMultiplier", 10.0),
                #         ("OnlyTransparent", False),
                #         ("CurrencyID", "NONE"),
                #     )
                strategy_params = {
                    "VolAvgLength": int(VolAvgParam),
                    "TriggerVolumeMultiplier": TriggerVolParam,
                    "CurrencyID": currency,
                    "OnlyTransparent": transparent,
                }
                cerebro.addstrategy(VolumeStrategy, **strategy_params)
                # no commission
                # cerebro.broker.setcommission(commission=0.005) # 0.5% commission
                cerebro.broker.setcash(1000000)
                cerebro.run()
                temp_df = pd.read_pickle(temp_save_path)
                # print(temp_df.head())
                # add to list of df
                df_list.append(deepcopy(temp_df))


    # save temp_df by appending to more permanent df
    if len(df_list) > 0:
        concat_df = type_raw_dataframe(pd.concat(df_list))
        archive_df = archive_df.append(deepcopy(concat_df), ignore_index=True)
    
    completed_currencies.append(currency)
    with open(results_folder.joinpath(f"completed_currencies.pkl"), "wb") as f:
        pickle.dump(completed_currencies,f)

    if len(completed_currencies)%10 == 0:
        break

# save archive_df
archive_df.to_pickle(archive_save_path)
# archive_df.to_excel(Path("../bt_data_runs/output.xlsx"), index=False)



Currencies:   0%|          | 0/11130 [00:00<?, ?it/s]

VolAvgParameters-BBYCAT:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BBYXRP:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BBZB:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BC:   0%|          | 0/18 [00:00<?, ?it/s]

...

VolAvgParameters-BCASH:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BCASH2:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BCC:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BCCX:   0%|          | 0/18 [00:00<?, ?it/s]

VolAvgParameters-BCD:   0%|          | 0/18 [00:00<?, ?it/s]

In [31]:
def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
             width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

        from backtrader import plot
        if cerebro.p.oldsync:
            plotter = plot.Plot_OldSync(**kwargs)
        else:
            plotter = plot.Plot(**kwargs)

        figs = []
        for stratlist in cerebro.runstrats:
            for si, strat in enumerate(stratlist):
                rfig = plotter.plot(strat, figid=si * 100,
                                    numfigs=numfigs, iplot=iplot,
                                    start=start, end=end, use=use)
                figs.append(rfig)

        for fig in figs:
            for f in fig:
                f.savefig(file_path, bbox_inches='tight')
        return figs

saveplots(cerebro, file_path = 'results/savefig.png') #run it

<IPython.core.display.Javascript object>

[[<Figure size 2000x1000 with 5 Axes>]]

In [11]:
import datetime
temp_df = pd.read_pickle(Path("../downloaded_data/nomics/candles/MATIC.pkl").resolve())

print(temp_df.head())

   timestamp      open      high       low     close       volume  \
0 2019-04-26  0.010115  0.010448  0.004777  0.005409  40593812.79   
1 2019-04-27  0.005397  0.005834  0.005109  0.005199   9347128.31   
2 2019-04-28  0.005192  0.005223  0.004319  0.004416   7700769.87   
3 2019-04-29  0.004374  0.004498  0.003424  0.003473   7180093.28   
4 2019-04-30  0.003462  0.005160  0.003462  0.004492  23813589.58   

   transparent_open  transparent_high  transparent_low  transparent_close  \
0          0.010115          0.010448         0.004777           0.005409   
1          0.005397          0.005834         0.005109           0.005199   
2          0.005192          0.005223         0.004319           0.004416   
3          0.004374          0.004498         0.003424           0.003473   
4          0.003462          0.005160         0.003462           0.004492   

   transparent_volume  volume_transparency.A  volume_transparency.C  \
0         40593812.79             40593812.0       

,buy_date,sell_date,buy_price,sale_price,currency_id,run_name,status


C:\Users\Gamer\PythonProjects\crypto-vol-scanner\bt_data_runs
C:\Users\Gamer\PythonProjects\crypto-vol-scanner\bt_data_runs\_temp_df.pkl


In [8]:
the_df = pd.read_pickle(Path("../downloaded_data/nomics/candles/BLAST4.pkl").resolve())
print(the_df)

    timestamp          open          high           low         close  \
0  2021-10-13  6.367930e-13  8.441230e-12  6.367930e-13  6.788568e-12   
1  2021-10-14  6.878518e-12  3.074635e-11  6.878518e-12  2.945999e-11   
2  2021-10-15  2.920287e-11  2.979936e-11  1.141983e-11  1.197093e-11   
3  2021-10-16  1.196746e-11  2.445129e-11  1.167031e-11  1.362400e-11   
4  2021-10-17  1.377348e-11  1.383726e-11  8.628017e-12  8.628017e-12   
5  2021-10-18  8.772985e-12  2.383518e-11  8.265677e-12  1.771344e-11   
6  2021-10-19  1.771231e-11  4.417751e-11  1.502659e-11  3.730057e-11   
7  2021-10-20  3.635111e-11  4.690438e-11  3.295748e-11  3.673870e-11   
8  2021-10-21  3.630924e-11  5.074858e-11  2.611744e-11  5.074858e-11   
9  2021-10-22  5.088413e-11  7.512333e-11  4.817824e-11  7.070740e-11   
10 2021-10-23  7.008855e-11  1.488976e-10  5.894845e-11  9.840697e-11   
11 2021-10-24  9.892187e-11  4.468625e-10  9.171280e-11  3.900700e-10   
12 2021-10-25  3.907073e-10  4.125335e-10  2.453118

In [9]:
len(the_df)

51

In [10]:
archive_df.to_excel(Path("../bt_data_runs/output.xlsx"), index=False)